# 7 網頁版聊天程式與文字生圖 Image API

## 7-1 準備工作


**準備工作**

請依照第 2 章說明上傳 .env 檔

In [ ]:
# 設定環境變數
!pip install python-dotenv openai googlesearch-python
from dotenv import load_dotenv
load_dotenv()

**從 github 下載套件**

In [ ]:
!git clone https://github.com/FlagTech/flagchat.git  flagchat

**匯入相關函式與資料**

In [ ]:
from flagchat import (
    get_reply,        # 輸入訊息串列傳回回覆
    chat,             # 輸入 system, user 發言取得回覆
                      # 並記錄對答歷史
    func_table,       # 可用工具函式參考表, 預設有 Google 搜尋函式
    set_backtrace,    # 設定記錄幾組對答 (預設：2)
    empty_history,    # 清除對答歷史
    set_verify_depth, # 設定驗證次數 (預設：3)
)

使用方法可參考 [Colab 範例筆記本](https://colab.research.google.com/drive/1FNoXyDCTld8vMJ1uF2x0O4eLlHVLDxEx?usp=sharing)

## 7-2 使用 gradio 套件快速建立網頁程式

### 安裝與使用 gradio

In [ ]:
!pip install gradio
import gradio as gr

建立基本的網頁介面

In [ ]:
web_chat = gr.Interface(
    fn = chat,
    inputs = ['text', 'text'],
    outputs = ['text']
)

In [ ]:
web_chat.queue()
web_chat.launch(share=True)

In [ ]:
web_chat.close()

### 使用串流方式顯示輸出

In [ ]:
web_chat = gr.Interface(
    fn = chat,
    inputs = ['text', 'text', 'checkbox'],
    outputs = ['text']
)

In [ ]:
web_chat.queue()
web_chat.launch()

In [ ]:
web_chat.close()

利用包裝函式組合片段內容

In [ ]:
def wrapper_chat(sys_msg, user_msg, stream):
    reply = ''
    for chunk in chat(sys_msg, user_msg, stream):
        reply += chunk
        yield reply

In [ ]:
web_chat = gr.Interface(
    fn = wrapper_chat,
    inputs = ['text', 'text', 'checkbox'],
    outputs = ['text']
)

In [ ]:
web_chat.queue()
web_chat.launch()

### 客製使用者介面

In [ ]:
messages = []

def wrapper_chat_bot(sys_msg, user_msg, stream):
    messages.append([user_msg, ''])
    for chunk in chat(sys_msg, user_msg, stream):
        messages[-1][1] += chunk
        yield messages

In [ ]:
web_chat = gr.Interface(
    fn=wrapper_chat_bot,
    inputs=[
        gr.Textbox(label='系統角色', value='使用繁體中文的小助理'),
        gr.Textbox(label='使用者發言'),
        gr.Checkbox(label='使用串流', value=False)],
    outputs=[gr.Chatbot(label='AI 回覆')]
)

In [ ]:
web_chat.queue()
web_chat.launch()

In [ ]:
web_chat.close()

## 7-3 使用 DALL‧E 的 Image API

### Image API 用法

In [ ]:
import openai

In [ ]:
openai.Image.create(            # 文字生圖
    prompt='夕陽下駛過海邊的火車',  # 描述文字
    n=1,                        # 生圖張數
    size='1024x1024')           # 影像大小

### 建立文字生圖像網址的函式

In [ ]:
def txt_to_img_url(prompt):
    response = openai.Image.create(
        prompt=prompt,
        n=1,
        size='1024x1024')
    return response['data'][0]['url']

In [ ]:
txt_to_img_url('田邊騎著腳踏車晃的少年')

In [ ]:
func_table.append(
    {                    # 每個元素代表一個函式
        "chain": False,  # 生圖後不需要傳回給 API
        "func": txt_to_img_url,
        "spec": {        # function calling 需要的函式規格
            "name": "txt_to_img_url",
            "description": "可由文字生圖並傳回圖像網址",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {
                        "type": "string",
                        "description": "描述要產生圖像內容的文字",
                    }
                },
                "required": ["prompt"],
            },
        }
    }
)

In [ ]:
for chunk in chat('小助理', '我想要夕陽下海豚躍出海面的圖像', True):
    print(chunk)

套用生圖功能到網頁聊天中

In [ ]:
messages = []

def wrapper_chat_bot(sys_msg, user_msg, stream):
    messages.append([user_msg, ''])
    for chunk in chat(sys_msg, user_msg, stream):
        messages[-1][1] += chunk
        yield messages

web_chat = gr.Interface(
    fn=wrapper_chat_bot,
    inputs=[
        gr.Textbox(label='系統角色', value='使用繁體中文的小助理'),
        gr.Textbox(label='使用者發言'),
        gr.Checkbox(label='使用串流', value=False)],
    outputs=[gr.Chatbot(label='AI 回覆')]
)

web_chat.queue()
web_chat.launch()

結果只會顯示連結, 對於 Chatbot 介面要提供 Mardown 格式才會顯示。

In [ ]:
web_chat.close()

### 包裝成生成 markdown 語法的函式


In [ ]:
def txt_to_img_md(prompt):
    return f'![{prompt}]({txt_to_img_url(prompt)})'

In [ ]:
func_table.pop()
func_table.append(
    {                    # 每個元素代表一個函式
        "chain": False,  # 生圖後不需要傳回給 API
        "func": txt_to_img_md,
        "spec": {        # function calling 需要的函式規格
            "name": "txt_to_img_md",
            "description": "可由文字生圖並傳回 markdown 圖像元素",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {
                        "type": "string",
                        "description": "描述要產生圖像內容的文字",
                    }
                },
                "required": ["prompt"],
            },
        }
    }
)

In [ ]:
messages = []

def wrapper_chat_bot(sys_msg, user_msg, stream):
    messages.append([user_msg, ''])
    for chunk in chat(sys_msg, user_msg, stream):
        messages[-1][1] += chunk
        yield messages

web_chat = gr.Interface(
    fn=wrapper_chat_bot,
    inputs=[
        gr.Textbox(label='系統角色', value='使用繁體中文的小助理'),
        gr.Textbox(label='使用者發言'),
        gr.Checkbox(label='使用串流', value=False)],
    outputs=[gr.Chatbot(label='AI 回覆')]
)

web_chat.queue()
web_chat.launch()

In [ ]:
web_chat.close()

In [ ]:
messages = []

def wrapper_chat_bot(sys_msg, user_msg, stream, model):
    messages.append([user_msg, ''])
    for chunk in chat(sys_msg, user_msg, stream, model=model):
        messages[-1][1] += chunk
        yield messages

web_chat = gr.Interface(
    fn=wrapper_chat_bot,
    inputs=[
        gr.Textbox(label='系統角色', value='使用繁體中文的小助理'),
        gr.Textbox(label='使用者發言'),
        gr.Checkbox(label='使用串流', value=False),
        gr.Radio(label='模型', choices=['gpt-3.5-turbo',
                                       'gpt-4'],
                 value='gpt-3.5-turbo')],
    outputs=[gr.Chatbot(label='AI 回覆')]
)

web_chat.queue()
web_chat.launch()

In [ ]:
web_chat.close()